### Piplelines
It chains together multiple steps so that the output of each step is used as input to the next step it makes easy to apply the same preprocessing to train and test
Euta step ko output act as a input of another model 

In [150]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [152]:
df = pd.read_csv('train.csv')

In [154]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [156]:
df.shape

(891, 12)

In [158]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [160]:
df.head()
df.shape

(891, 8)

In [162]:
# step1 = train_test_split 

X_train , X_test , y_train , y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)

In [166]:
X_train.head()
X_train.shape

(712, 7)

In [168]:
X_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
709,3,male,NaN,1,1,15.2458,C
439,2,male,31.0,0,0,10.5000,S
840,3,male,20.0,0,0,7.9250,S
720,2,female,6.0,0,1,33.0000,S
39,3,female,14.0,1,0,11.2417,C


In [170]:
y_train.head()

331    0
733    0
382    0
704    0
813    0
Name: Survived, dtype: int64

In [197]:
# aba sex lai hamle order ma garna mildena so we use nominal encoding ie hotcodeencoder
df.isnull().sum()
X_train.shape

(712, 7)

In [174]:
# applying imputation handling missing values
si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')
X_train_age = si_age.fit_transform(X_train[['Age']])
X_train_embarked = si_embarked.fit_transform(X_train[['Embarked']])

X_test_age = si_age.fit_transform(X_test[['Age']])
X_test_embarked = si_embarked.fit_transform(X_test[['Embarked']])

In [199]:
X_test_age.shape

(179, 1)

In [177]:
# one hot encoding sex and embarked
ohe = OneHotEncoder(sparse_output=False)
ohe1 = OneHotEncoder(sparse_output=False)
X_train_sex = ohe.fit_transform(X_train[['Sex']] )
X_train_embarked = ohe1.fit_transform(X_train_embarked)
X_test_sex = ohe.fit_transform(X_test[['Sex']] )
X_test_embarked = ohe1.fit_transform(X_test_embarked)
X_train_sex




array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [179]:
X_train_embarked

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [181]:
# how to add new columns in the given dataframe
X_train_rem = X_train.drop(columns=['Sex','Age','Embarked'])

In [183]:
X_test_rem = X_test.drop(columns=['Sex','Age','Embarked'])

In [185]:
X_train_transformed = np.concatenate ((X_train_rem , X_train_age , X_train_sex , X_train_embarked) , axis=1)
X_test_transformed = np.concatenate ((X_test_rem , X_test_age , X_test_sex , X_test_embarked) , axis=1)

In [201]:
X_test_transformed.shape

(179, 10)

In [203]:
clf = DecisionTreeClassifier()
clf.fit(X_train_transformed , y_train)

DecisionTreeClassifier()

In [215]:
y_pred = clf.predict(X_test_transformed)
y_pred.shape

(179,)

In [217]:
y_test.shape

(179,)

In [219]:
from sklearn.metrics import accuracy_score

In [221]:
accuracy_score(y_test , y_pred)

0.7374301675977654

In [223]:
import pickle 

In [245]:
import os
import pickle

# Create the directory if it doesn't exist
os.makedirs('models', exist_ok=True)

# Now save your files
pickle.dump(ohe, open('models/ohe.pkl','wb'))
pickle.dump(ohe1, open('models/ohe1.pkl','wb'))
pickle.dump(clf, open('models/clf.pkl','wb'))